In [ ]:
import os

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
class CFG:
    num_workers = 4
    path = "../input/patent-na-rozum-dbl/"  
    path1 = "../input/foreles/mel/"  
    path2 = "../input/mybert/myb/"
    path2a = "../input/myparqs/wups/"
    path3 = "../input/bert768/ab768/"
    path4 = "../input/dblina/"
    path4a = "../input/split10f/xl/xl/"
    path0 = "../input/sgkf8dbl/sgkf/"
    config_path1 = path1 +'config.pth'
    config_path2 = path2 +'config.pth'
    config_path2a = path2a +'config.pth'
    config_path3 = path3 +'config.pth'
    config_path4 = path4 +'config.pth'
    config_path4a = path4a +'config.pth'
    config_path0 = path0 +'config.pth'
    model1 = "google/electra-large-discriminator"
    model2 = "anferico/bert-for-patents"
    model2a = "microsoft/deberta-v3-large"
    model3 = "anferico/bert-for-patents"
    model4 = "microsoft/deberta-v3-large"
    model4a = "microsoft/deberta-v2-xlarge"
    model0 = "microsoft/deberta-v3-large"
    weights1 = "../input/elektra/"
    weights2 = "../input/berties/"
    weights2a = "../input/myparqs/wups/"
    weights3 = "../input/berties7/"
    weights4 = "../input/dblina/"
    weights4a = "../input/split10f/xl/xl/"
    weights0 = "../input/sgkf8dbl/sgkf/"
    weights0a = "../input/normdbl/"
    batch_size = 32
    fc_dropout = 0.2
    target_size = 1
    max_len = 133
    seed = 56
    n_fold1 = 9
    trn_fold1 = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    n_fold2 = 8
    trn_fold2 = [0, 1, 2, 3, 4, 5, 6, 7]
    n_fold2a = 5
    trn_fold2a = [0, 1, 2, 3, 4]
    n_fold3 = 5
    trn_fold3 = [0, 1, 2, 3, 4]
    n_fold4 = 4
    trn_fold4 = [0, 1, 2, 3,]
    n_fold4a = 3
    trn_fold4a = [0, 1, 2]
    n_fold0 = 8
    trn_fold0 = [0, 1, 2, 3, 4, 5, 6, 7]  
    n_fold0a = 5
    trn_fold0a = [0, 1, 2, 3, 4]  

In [ ]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

In [ ]:
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/mojekola/mk transformers')
os.system('python -m pip install --no-index --find-links=../input/mojekola/mk tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=56):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=56)

In [ ]:
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())

In [ ]:
cpc_texts = torch.load(CFG.path0+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

In [ ]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

In [ ]:
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path0+'tokenizer/')

In [ ]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text, add_special_tokens=True, max_length=cfg.max_len,
                           padding="max_length", return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs


In [ ]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(nn.Linear(self.config.hidden_size, 512),
                                       nn.Tanh(),
                                       nn.Linear(512, 1),
                                       nn.Softmax(dim=1) )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [ ]:
class Normal(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(nn.Linear(self.config.hidden_size, 512),
                                       nn.Tanh(),
                                       nn.Linear(512, 1),
                                       nn.Softmax(dim=1) )
        self.layer_norm1 = nn.LayerNorm(self.config.hidden_size)
        self._init_weights(self.attention)
        self.linear = nn.Linear(self.config.hidden_size, 1)
           
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_state = outputs[0]
        input_mask_expanded = inputs["attention_mask"].unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        out = sum_embeddings / sum_mask
        out = self.layer_norm1(out)
        output = self.fc(out)
        return output

In [ ]:
class CustomModel768(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc1 = nn.Linear(self.config.hidden_size, 768)
        self.fc2 = nn.Linear(768,1)
        self._init_weights(self.fc1)
        self._init_weights(self.fc2)
        self.layer_norm1 = nn.LayerNorm(self.config.hidden_size)
        
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
        
    def forward(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_state = outputs[0]
        input_mask_expanded = inputs["attention_mask"].unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        out = sum_embeddings / sum_mask
        out = self.layer_norm1(out)
        out = self.fc1(out)
        out = self.fc2(out)
        
        return out

In [ ]:
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [ ]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions0 = []

In [ ]:
for fold in CFG.trn_fold0:
    model = CustomModel(CFG, config_path=CFG.config_path0, pretrained=False)
    state = torch.load(CFG.weights0+f"{CFG.model0.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions0.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
    
for fold in CFG.trn_fold0a:
    model = Normal(CFG, config_path=CFG.config_path0, pretrained=False)
    state = torch.load(CFG.weights0a+f"{CFG.model0.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions0.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()    
    
predictions0 = np.mean(predictions0, axis=0)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LOGGER = get_logger()
seed_everything(seed=56)

In [ ]:
cpc_texts = torch.load(CFG.path1+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

In [ ]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

In [ ]:
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path1+'tokenizer/')

test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions1 = []

In [ ]:
for fold in CFG.trn_fold1:
    model = Normal(CFG, config_path=CFG.config_path1, pretrained=False)
    state = torch.load(CFG.weights1+f"gel_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions1.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions1 = np.mean(predictions1, axis=0)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LOGGER = get_logger()
seed_everything(seed=56)

In [ ]:
test = pd.read_csv(INPUT_DIR+'test.csv')

cpc_texts = torch.load(CFG.path2+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

In [ ]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

In [ ]:
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path2+'tokenizer/')

In [ ]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions2 = []

In [ ]:
for fold in CFG.trn_fold2:
    model = Normal(CFG, config_path=CFG.config_path2, pretrained=False)
    state = torch.load(CFG.weights2+f"ab-patents_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions2.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions2 = np.mean(predictions2, axis=0)

In [ ]:
test = pd.read_csv(INPUT_DIR+'test.csv')

cpc_texts = torch.load(CFG.path2a+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

In [ ]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

In [ ]:
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path2a+'tokenizer/')

test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions2a = []

In [ ]:
for fold in CFG.trn_fold2a:
    model = Normal(CFG, config_path=CFG.config_path2a, pretrained=False)
    state = torch.load(CFG.weights2a + f"{CFG.model2a.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions2a.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()    
    
predictions2a = np.mean(predictions2a, axis=0)


In [ ]:
submission['score2a'] = predictions2a         
submission['score2b'] = predictions2
submission['score2']= 0.35*submission['score2b']+0.65*submission['score2a']
del submission['score2a']
del submission['score2b']

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LOGGER = get_logger()
seed_everything(seed=56)

In [ ]:
cpc_texts = torch.load(CFG.path3+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

In [ ]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

In [ ]:
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path3+'tokenizer/')

In [ ]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions3 = []

In [ ]:
for fold in CFG.trn_fold3:
    model = CustomModel768(CFG, config_path=CFG.config_path3, pretrained=False)
    state = torch.load(CFG.weights3+f"ab768-patents_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions3.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions3 = np.mean(predictions3, axis=0)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LOGGER = get_logger()
seed_everything(seed=56)

In [ ]:
cpc_texts = torch.load(CFG.path4+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

In [ ]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

In [ ]:
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path4+'tokenizer/')

In [ ]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions4 = []

In [ ]:
for fold in CFG.trn_fold4:
    model = CustomModel(CFG, config_path=CFG.config_path4, pretrained=False)
    state = torch.load(CFG.weights4+f"{CFG.model4.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions4.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions4 = np.mean(predictions4, axis=0)

In [ ]:
test = pd.read_csv(INPUT_DIR+'test.csv')

cpc_texts = torch.load(CFG.path4a+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

In [ ]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

In [ ]:
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path4a+'tokenizer/')

In [ ]:
predictions4a = []

test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

for fold in CFG.trn_fold4a:
    model = CustomModel(CFG, config_path=CFG.config_path4a, pretrained=False)
    state = torch.load(CFG.weights4a + f"xlarge_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions4a.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
    
predictions4a = np.mean(predictions4a, axis=0)

In [ ]:
submission['score4b'] = predictions4         
submission['score4a'] = predictions4a
submission['score4']= 0.56*submission['score4b']+0.44*submission['score4a']
del submission['score4b']
del submission['score4a']

In [ ]:
submission['score0'] = predictions0
submission['score1'] = predictions1
#submission['score2'] = predictions2
submission['score3'] = predictions3         
#submission['score4'] = predictions4
submission['scoreB']=0.20*submission['score0']+0.40*submission['score1']+0.12*submission['score3']+0.28*submission['score4']
del submission['score0']
del submission['score1']
#del submission['score2']
del submission['score3']
del submission['score4']

submission['score']= 0.56*submission['scoreB'] + 0.44*submission['score2']
del submission['score2']
del submission['scoreB']
display(submission.head())

In [ ]:
submission[['id', 'score']].to_csv('submission.csv', index=False)